In [220]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

cache_dir = LocalFileStore("./.cache/")
llm = ChatOpenAI(temperature=0.1)

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n", 
    chunk_size=600,
    chunk_overlap=100,
)

embedder = OpenAIEmbeddings()
cached_embedder = CacheBackedEmbeddings.from_bytes_store(embedder, cache_dir)

loarder = UnstructuredFileLoader("./files/novel.txt")
docs = loarder.load_and_split(text_splitter=splitter)
vector_store = FAISS.from_documents(docs, cached_embedder)

retriever = vector_store.as_retriever()
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 답변 도우미 입니다. 모르는 정보는 지어내지 마세요. 아래 문맥에 따라 답변해 주세요.\n\n {context}"),
    ("human", "{question}")
])

chain = { "context": retriever, "question": RunnablePassthrough() } | prompt | llm

chain.invoke("김첨지의 아내는 어떻게 되나요")

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "김첨지의 아내는 어떻게 되나요"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel] Entering Chain run with input:
{
  "input": "김첨지의 아내는 어떻게 되나요"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 4:chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "김첨지의 아내는 어떻게 되나요"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 4:chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "김첨지의 아내는 어떻게 되나요"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel] s] Exiting Chain run with output:
[outputs]
[chain/start] [1:chain:RunnableSequence > 5:prompt:ChatPromptTemplate] Entering Prompt run with input:
[inputs]
[chain/end] [1:chain:RunnableSequence > 5:prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "chat",
    "ChatPrompt

AIMessage(content='김첨지의 아내는 아픈 상태로 집에 붙어 있고, 김첨지는 아내를 두고 외출하려다가 아내의 부탁에 마음이 켕겨서 집을 나가지 않았습니다. 아내는 아픔을 호소하며 김첨지에게 집에 있어 달라고 부탁했습니다.')